# Traffic sign recognition

#### 1. Import libraries

In [1]:
import os
import sys
assert sys.version_info >= (3,7)
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import time
import imutils
import random
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

!pip install numpy

#### 2. Setup

In [2]:
# For reproducibility,
np.random.seed(99)

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)
cv.useOptimized()

True

#### 3. Declare all modifiable variables for easier tuning

In [3]:
# Traffic signs with blue colour
blue_ts = (20,21,22,23,24,25,26,27,28,29,30,31)

# Traffic signs with red colour
red_ts = (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,33,52,53,54,55,56,57)

# Traffic signs with yellow colour
yellow_ts = (18,19,32,34,335,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51)

# All traffic signs
all_ts = tuple(range(0, 58))

# Bounderies for HSV thresholding, fine tuned to fit the dataset
# Red colour segmentation
lower_red, upper_red = (0, 25, 10), (10, 255, 255)
lower_red2, upper_red2 = (150, 25, 0), (180, 255, 255)

# Blue colour segmentation
lower_blue, upper_blue = (90, 60, 10), (135, 255, 255)

# Yellow colour segmentation
lower_yellow, upper_yellow = (10, 50, 10), (55, 255, 255)
lower_black, upper_black = (0, 0, 0), (180, 50, 80)

#### 4. Create reusable functions

In [4]:
def hsv_edges_bg_mask(image):
    image = cv.GaussianBlur(image, (5,5), 3)
    h, w = image.shape[:2]
    noise_mask = np.ones((h+2, w+2))
    
    
    # Get the position to fill
    left = int(w*0.1)
    right = int(w*0.9)
    top = int(h*0.1)
    bottom = int(h*0.9)
    
    
    # Get colour of edges
    top_left_colour = image[top,left]
    bottom_left_colour = image[bottom, left]
    top_right_colour = image[top,right]
    bottom_right_colour = image[bottom,right]
    
    
    # Get all the noise of the background
    mask = cv.inRange(image, top_left_colour - 7, top_left_colour + 7)
    mask += cv.inRange(image, bottom_left_colour - 7, bottom_left_colour + 7)
    mask += cv.inRange(image, top_right_colour - 7, top_right_colour + 7)
    mask += cv.inRange(image, bottom_right_colour - 7, bottom_right_colour + 7)
    
    
    return mask

In [5]:
def canny_seg(image, dilate_kernel1, dilate_kernel2, dilate_iterations):    
    # Smoothing the image
    image = cv.GaussianBlur(image, (7, 7), 1)
    
    
    # Canny segmentation
    mask = cv.Canny(image, 1, 100)
    
    
    # Dilate the edges found by canny segmentation
    mask = cv.dilate(mask, (dilate_kernel1,dilate_kernel2), iterations=dilate_iterations)
    
    
    return mask

In [6]:
def blue_seg(image_hsv, image_bgr, background_mask):
    # Thresholding
    mask = cv.inRange(image_hsv, lower_blue, upper_blue)
    
    # Removing edges
    canny_mask = canny_seg(image_bgr, 3, 3, 2)
    
    
    final_mask = mask - canny_mask - background_mask
    
    
    return final_mask, mask, canny_mask

In [7]:
def red_seg(image_hsv, image_bgr, background_mask):
    # Thresholding
    mask = cv.inRange(image_hsv, lower_red, upper_red)
    mask += cv.inRange(image_hsv, lower_red2, upper_red2)
    
    
    # Removing edges
    canny_mask = canny_seg(image_bgr, 1, 1, 1)
    
    
    final_mask = mask - canny_mask - background_mask
    
    
    return final_mask, mask, canny_mask

In [8]:
def black_seg(image_hsv, image_bgr, background_mask):
    h,w = image_hsv.shape[:2]
    
    
    # Segment the edges
    canny_mask = canny_seg(image_bgr, 1, 1, 1)

    
    # edged is the edge detected image
    cnts = cv.findContours(canny_mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv.contourArea, reverse = True)[:5]
    # loop over the contours
    i = 0
    for c in cnts:
        if (cv.contourArea(c) < h * w * 0.10):
            break

        # approximate the contour
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.1 * peri, True)
        # Contour has 3 points, represents a triangle
        if len(approx) == 3:
            temp_mask = np.zeros(image_hsv.shape[:2], dtype = np.uint8)
            cv.drawContours(temp_mask, cnts, i, (255,255,255), -1)
            return temp_mask - background_mask, temp_mask, canny_mask
        i+=1
    
    
    mask = cv.inRange(image_hsv, lower_black, upper_black)
    mask += cv.inRange(image_hsv, lower_yellow, upper_yellow)
    
    
    final_mask = mask - canny_mask - background_mask
    
    
    return final_mask, mask, canny_mask

In [9]:
def find_largest_contour(image, fill = -1):
    # Threshold to keep only the bright area in the mask 
    _, image = cv.threshold(image, 200, 255, cv.THRESH_TOZERO)
    
    mask = np.zeros(image.shape, dtype = np.uint8)
    largest_contour = None
    
    
    # Find all contours
    if (int(cv.__version__[0]) > 3):
        contours, hierarchy = cv.findContours(image, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    else:
        _, contours, hierarchy = cv.findContours(image, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    
    
    # Find and draw largest contour
    if len(contours) != 0:
        cnt_list = np.zeros(len(contours))
        for i in range(0,len(contours)):
            cnt_list[i] = cv.contourArea(contours[i])

            
        largest_contour_index = np.argmax(cnt_list)
        largest_contour = contours[largest_contour_index]
        cv.drawContours(mask, contours, largest_contour_index, (255,255,255), fill)

        
    return largest_contour, mask

In [10]:
def draw_bounding_box(image, largest_contour, colour):
    bounding_box = image.copy()
    
    
    # If contour is not exists
    if (largest_contour is None):
        return bounding_box, (0,0,0,0)
    
    
    cv.boundingRect(largest_contour)
    x,y,w,h = cv.boundingRect(largest_contour)
    cv.rectangle(bounding_box,(x,y),(x+w,y+h),colour,2)
    return bounding_box, (x,y,x+w,y+h)

In [11]:
def closest_contour(contours, image):
    h, w = image.shape[:2]
    center = np.array((int(h/2), int(w/2)))
    rect = np.zeros((h, w), dtype = np.uint8)
    
    
    closest = 0
    closest_distance = 9999
    
    
    if(contours[0] is not None):
        blue_dist = cv.moments(contours[0])
        cx = int(blue_dist['m10']/max(blue_dist['m00'], 1))
        cy = int(blue_dist['m01']/max(blue_dist['m00'], 1))
        blue_distance = ((cx - center[1])**2 + (cy - center[0])**2)**0.5
        if blue_distance < closest_distance:
            _,_,w0,y0 = cv.boundingRect(contours[0])
            if(w0*y0 > h*w*0.2):
                closest_distance = blue_distance
    
    
    if(contours[1] is not None):
        red_dist = cv.moments(contours[1])
        cx = int(red_dist['m10']/max(red_dist['m00'], 1))
        cy = int(red_dist['m01']/max(red_dist['m00'], 1))
        red_distance = ((cx - center[1])**2 + (cy - center[0])**2)**0.5
        if red_distance < closest_distance:
            _,_,w1,y1 = cv.boundingRect(contours[1])
            if(w1*y1 > h*w*0.2):
                closest = 1
                closest_distance = red_distance
        
        
    if(contours[2] is not None):
        yellow_dist = cv.moments(contours[2])
        cx = int(yellow_dist['m10']/max(yellow_dist['m00'], 1))
        cy = int(yellow_dist['m01']/max(yellow_dist['m00'], 1))
        yellow_distance = ((cx - center[1])**2 + (cy - center[0])**2)**0.5
        if yellow_distance < closest_distance:
            _,_,w2,y2 = cv.boundingRect(contours[2])
            if(w2*y2 > h*w*0.2):
                closest = 2
        
    
    return closest

In [12]:
def shape_detection(image):
    h,w = image.shape[:2]
    mask = np.zeros((h, w), dtype = np.uint8)

    
    # Segment the edges
    canny_mask = canny_seg(image, 1, 1, 1)
    
    
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    # detect circles
    circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, 100, param1 = 50, param2 = 25, minRadius=int(w*0.2), maxRadius=int(w*0.4))

    
    # draw largest circle
    if circles is not None:
        circles=sorted(circles[0],key=lambda x:x[2],reverse=True)
        circles = np.uint8(np.around(circles))
        for i in circles:
            center = (i[0], i[1])
            # circle outline
            radius = i[2]
            cv.circle(mask, center, radius, 255, 2)
    
    final_mask = mask
    
    
    return final_mask, mask, canny_mask

In [13]:
def segmentation(annotation, images, traffic_sign, save_result):
    seconds = time.time()
    ts = np.where(annotation['Category'].isin(traffic_sign))[0]
    combine_all = np.empty((150,1050,3))
    bounding_box_points = []
    
    for i in range (len(ts)):
        # Create a copy of the original image
        image = images[ts[i]].copy()
        h, w = image.shape[:2]
        
        
        # Convert image into hsv colour space and gray colour space
        image_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

        
        # Segment background colour
        background_mask = hsv_edges_bg_mask(image_hsv)
 
        if True:
            # HSV segmentation
            blueseg_mask, blue_mask, blue_canny_mask = blue_seg(image_hsv, image, background_mask)
            redseg_mask, red_mask, red_canny_mask = red_seg(image_hsv, image, background_mask)
            yellowseg_mask, yellow_mask, yellow_canny_mask = black_seg(image_hsv, image, background_mask)


            # Find largest contour
            blue_largest_contour, blue_largest_contour_mask = find_largest_contour(blueseg_mask)
            red_largest_contour, red_largest_contour_mask = find_largest_contour(redseg_mask)
            yellow_largest_contour, yellow_largest_contour_mask = find_largest_contour(yellowseg_mask)


            # Draw bounding box
            blue_bounding_box, blue_box_points = draw_bounding_box(image, blue_largest_contour, (0,255,0))
            red_bounding_box, red_box_points = draw_bounding_box(image, red_largest_contour, (0,255,0))
            yellow_bounding_box, yellow_box_points = draw_bounding_box(image, yellow_largest_contour, (0,255,0))


            # Find the contour the is closest to the center
            contours = [blue_largest_contour, red_largest_contour, yellow_largest_contour]
            closest = closest_contour(contours, image)


            if(closest == 0):
                canny_mask = blue_canny_mask
                hsv_mask = blue_mask
                segmented_mask = blueseg_mask
                final_mask = blue_largest_contour_mask
                bounding_box = blue_bounding_box
                bounding_box_points.append(blue_box_points)
            elif (closest == 1):
                canny_mask = red_canny_mask
                hsv_mask = red_mask
                segmented_mask = redseg_mask
                final_mask = red_largest_contour_mask
                bounding_box = red_bounding_box
                bounding_box_points.append(red_box_points)
            else:
                canny_mask = yellow_canny_mask
                hsv_mask = yellow_mask
                segmented_mask = yellowseg_mask
                final_mask = yellow_largest_contour_mask
                bounding_box = yellow_bounding_box
                bounding_box_points.append(yellow_box_points)
        
        else:
            shape_largest_contour, shape_largest_contour_mask = find_largest_contour(shapeseg_mask)
            shape_bounding_box, shape_box_points = draw_bounding_box(image, shape_largest_contour, (0,255,0))
            
            canny_mask = shape_canny_mask
            hsv_mask = shape_mask
            segmented_mask = shapeseg_mask
            
            
            final_mask = shape_largest_contour_mask
            bounding_box = shape_bounding_box
            bounding_box_points.append(shape_box_points)


        if(save_result):
            canny_mask = np.stack((canny_mask,)*3, axis=-1)
            background_mask = np.stack((background_mask,)*3, axis=-1)
            hsv_mask = np.stack((hsv_mask,)*3, axis=-1)
            segmented_mask = np.stack((segmented_mask,)*3, axis=-1)
            final_mask = np.stack((final_mask,)*3, axis=-1)
            combine = np.concatenate((image, canny_mask, background_mask, hsv_mask, segmented_mask, final_mask, bounding_box), axis = 1)
            combine = cv.resize(combine, (1050,150), interpolation = cv.INTER_AREA)
            
            combine = np.concatenate((np.zeros((30, 1050, 3)), combine), axis = 0)
            cv.putText(combine, ("original image"), (5, 15), 0, 0.5, (255,255,255), 1, cv.LINE_AA)
            cv.putText(combine, ("edges detected"), (155, 15), 0, 0.5, (255,255,255), 1, cv.LINE_AA)
            cv.putText(combine, ("background"), (305, 15), 0, 0.5, (255,255,255), 1, cv.LINE_AA)
            cv.putText(combine, ("HSV mask"), (455, 15), 0, 0.5, (255,255,255), 1, cv.LINE_AA)
            cv.putText(combine, ("segmentation"), (605, 15), 0, 0.5, (255,255,255), 1, cv.LINE_AA)
            cv.putText(combine, ("largest contour"), (755, 15), 0, 0.5, (255,255,255), 1, cv.LINE_AA)
            cv.putText(combine, ("bounding box"), (905, 15), 0, 0.5, (255,255,255), 1, cv.LINE_AA)
            
            combine_all = np.concatenate((combine_all, combine), axis = 0)      
            if (i%100 == 0):
                cv.imwrite('saved' + str(i//100) + '.jpg', combine_all)
                combine_all = combine
                print(i, "/", len(ts), "Elapsed time:", time.time() - seconds, "Average time:", (time.time() - seconds)/(i+1))
        elif (i%100 == 0):
            print(i, "/", len(ts), "Elapsed time:", time.time() - seconds, "Average time:", (time.time() - seconds)/(i+1))
    
    return bounding_box_points

#### 5. Load train annotation

In [14]:
# Path to the train set annotation txt file
path = os.path.join(os.getcwd(), 'TSRD-Train Annotation\\TsignRecgTrain4170Annotation.txt')

# Name of the columns
columns = ['File Name', 'Width', 'Height', 'Start_X', 'Start_Y', 'End_X', 'End_Y', 'Category']

# Read the content of the train annotation txt file into train_annotation
train_annotation = pd.read_csv(path, sep=";", index_col = False, names = columns)

# Category of the images
train_category = train_annotation['Category'].values

#### 6. Load train images

In [15]:
#path to the folder containing the trains set image files
path = os.path.join(os.getcwd(), 'TSRD-train')

#read the train images into train_image
train_images = []
for i in range (train_annotation.shape[0]):
    train_images.append(cv.imread(path + '\\' + train_annotation.iloc[i][0]))

#### 7. Segmentation

In [16]:
bounding_box_points = segmentation(train_annotation, train_images, all_ts, save_result = False)

0 / 4170 Elapsed time: 0.004998445510864258 Average time: 0.004998445510864258
100 / 4170 Elapsed time: 0.25797367095947266 Average time: 0.0025541947619749767
200 / 4170 Elapsed time: 0.44406962394714355 Average time: 0.002209301611677331
300 / 4170 Elapsed time: 0.6384623050689697 Average time: 0.0021211372261427564
400 / 4170 Elapsed time: 0.8795437812805176 Average time: 0.002193376013168373
500 / 4170 Elapsed time: 1.0844290256500244 Average time: 0.002164528993313422
600 / 4170 Elapsed time: 1.335904836654663 Average time: 0.0022228033887764778
700 / 4170 Elapsed time: 1.594414234161377 Average time: 0.002274485355437057
800 / 4170 Elapsed time: 1.8116679191589355 Average time: 0.0022617577018213928
900 / 4170 Elapsed time: 2.02583646774292 Average time: 0.0022484311517679467
1000 / 4170 Elapsed time: 2.3538689613342285 Average time: 0.002351517443890338
1100 / 4170 Elapsed time: 2.5972237586975098 Average time: 0.002358967991550872
1200 / 4170 Elapsed time: 2.7923078536987305 Av

#### 8. Segmentation Evaluation

In [17]:
total_overlapp = 0
area = 0
for i in range (len(bounding_box_points)):
    area1 = (bounding_box_points[i][2] - bounding_box_points[i][0]) * (bounding_box_points[i][3] - bounding_box_points[i][1])
    area2 = (train_annotation['End_X'][i] - train_annotation['Start_X'][i]) * (train_annotation['End_Y'][i] - train_annotation['Start_Y'][i])
    X_diff = max(0, min(bounding_box_points[i][2], train_annotation['End_X'][i]) - max(bounding_box_points[i][0], train_annotation['Start_X'][i]))
    Y_diff = max(0, min(bounding_box_points[i][3], train_annotation['End_Y'][i]) - max(bounding_box_points[i][1], train_annotation['Start_Y'][i]))
    overlapp = X_diff * Y_diff
    total_overlapp += overlapp
    area = area + area1 + area2 - overlapp
print("Segmentation accuracy for train set:", total_overlapp/area)

Segmentation accuracy for train set: 0.9010446792760016


#### 9. Split train and val set

In [18]:
# sort the samples according to class
sorted_array = sorted(enumerate(train_category), key=lambda x: x[1])
sorted_indices = [index for index, _ in sorted_array]

# split train:val according to the ratio
val_ratio = 0.1

# calculate number of val samples in each class
class_count = [0]
for i in range (len(train_category)):
    if(train_category[i] >= len(class_count)):
        class_count.append(0)
    class_count[train_category[i]] += 1
    
# split train and val set
train_set = []
val_set = []
random.seed(42)
prev_samples = 0
for i in range (len(class_count)):
    class_val_count = int(class_count[i] * val_ratio)
    val_set_index = []
    marked = [False] * class_count[i]
    for j in range(class_val_count):
        selected = random.randint(0, class_count[i] - 1)
        while (marked[selected]):
            selected += 1
            if selected == class_val_count:
                selected = 0
        marked[selected] = True
    for j in range (len(marked)):
        if(marked[j]):
            val_set.append(sorted_indices[prev_samples + j])
        else:
            train_set.append(sorted_indices[prev_samples + j])
    prev_samples += class_count[i]

# shuffle the train and val sets
random.shuffle(train_set)
random.shuffle(val_set)

# create train and val set samples and labels
train_set_images = []
train_set_cat = []
val_set_images = []
val_set_cat = []
for i in range(len(train_set)):
    ind = train_set[i]
    train_set_images.append(train_images[ind][train_annotation['Start_Y'][ind]:train_annotation['End_Y'][ind], train_annotation['Start_X'][ind]:train_annotation['End_X'][ind]])
    train_set_cat.append(train_category[ind])

for i in range(len(val_set)):
    ind = val_set[i]
    val_set_images.append(train_images[ind][bounding_box_points[ind][1]:bounding_box_points[ind][3], bounding_box_points[ind][0]:bounding_box_points[ind][2]])
    val_set_cat.append(train_category[ind])

#### 10. Feature extraction using HOG

In [19]:
winSize = (128, 128)
blockSize = (16, 16)
blockStride = (8, 8)
cellSize = (8, 8)
nbins = 9
hog = cv.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)

def feature_extraction(image):
    # preprocessing
    image_size = (128, 128)
    image = cv.resize(image, image_size)
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Compute HOG features
    features = hog.compute(gray_image)

    # Reshape the feature vector
    features = np.reshape(features, (-1))
    return features

#### 11. Train SVM classifier

In [20]:
# Extract Features
train_set_features = []
val_set_features = []
print("Extracting features for train set...")
for i in range(len(train_set_images)):
    train_set_features.append(feature_extraction(train_set_images[i]))
if len(val_set_images) > 0:
    print("Extracting features for val set...")
    for i in range(len(val_set_images)):
        val_set_features.append(feature_extraction(val_set_images[i]))
train_set_features = np.array(train_set_features)
val_set_features = np.array(val_set_features)

# Normalization
scaler = StandardScaler()
train_set_features = scaler.fit_transform(train_set_features)
if len(val_set_images) > 0:
    val_set_features = scaler.transform(val_set_features)

# Train the SVM classifier
print("Training classifier...")
classifier = svm.SVC()
classifier.fit(train_set_features, train_set_cat)

# get training accuracy
print("Predicting train set...")
predicted_train_classes = classifier.predict(train_set_features)
print("Calculating train set accuracy...")
train_accuracy = accuracy_score(train_set_cat, predicted_train_classes)
print("Correct prediction: ", int(len(predicted_train_classes) * train_accuracy), "/", len(predicted_train_classes))
print("Accuracy score: ", train_accuracy)

#get val accuracy
if len(val_set_images) > 0:
    print("\nPredicting val set...")
    predicted_val_classes = classifier.predict(val_set_features)
    print("Calculating val accuracy...")
    val_accuracy = accuracy_score(val_set_cat, predicted_val_classes)
    print("Correct prediction: ", int(len(predicted_val_classes) * val_accuracy), "/", len(predicted_val_classes))
    print("Accuracy score: ", val_accuracy)

Extracting features for train set...
Extracting features for val set...
Training classifier...
Predicting train set...
Calculating train set accuracy...
Correct prediction:  3776 / 3776
Accuracy score:  1.0

Predicting val set...
Calculating val accuracy...
Correct prediction:  371 / 394
Accuracy score:  0.9416243654822335


#### 12. Segmentation and classification evaluation with test set

In [21]:
# Path to the train set annotation txt file
path = os.path.join(os.getcwd(), 'TSRD-Test Annotation\\TsignRecgTest1994Annotation.txt')

# Name of the columns
columns = ['File Name', 'Width', 'Height', 'Start_X', 'Start_Y', 'End_X', 'End_Y', 'Category']

# Read the content of the train annotation txt file into train_annotation
test_annotation = pd.read_csv(path, sep=";", index_col = False, names = columns)

# Category of the images
test_category = test_annotation['Category'].values

#path to the folder containing the trains set image files
path = os.path.join(os.getcwd(), 'TSRD-test')

#read the train images into train_image
test_images = []
for i in range (test_annotation.shape[0]):
    test_images.append(cv.imread(path + '\\' + test_annotation.iloc[i][0]))

In [22]:
# segmentate test set
bounding_box_points = segmentation(test_annotation, test_images, all_ts, save_result = False)

0 / 1994 Elapsed time: 0.0029449462890625 Average time: 0.0029449462890625
100 / 1994 Elapsed time: 0.26861071586608887 Average time: 0.0026595120382781074
200 / 1994 Elapsed time: 0.5595118999481201 Average time: 0.0027836412932742294
300 / 1994 Elapsed time: 0.8564953804016113 Average time: 0.0028454996026631607
400 / 1994 Elapsed time: 1.193113088607788 Average time: 0.0029753443606179253
500 / 1994 Elapsed time: 1.5939459800720215 Average time: 0.003181528902339364
600 / 1994 Elapsed time: 1.9804935455322266 Average time: 0.003295330358622673
700 / 1994 Elapsed time: 2.3104562759399414 Average time: 0.0032959433322966355
800 / 1994 Elapsed time: 2.6508922576904297 Average time: 0.0033094784740205115
900 / 1994 Elapsed time: 3.0414483547210693 Average time: 0.0033756363537414753
1000 / 1994 Elapsed time: 3.38366961479187 Average time: 0.0033802893254664037
1100 / 1994 Elapsed time: 3.790205955505371 Average time: 0.003442512221167458
1200 / 1994 Elapsed time: 4.152076721191406 Avera

In [23]:
total_overlapp = 0
area = 0
for i in range (len(bounding_box_points)):
    area1 = (bounding_box_points[i][2] - bounding_box_points[i][0]) * (bounding_box_points[i][3] - bounding_box_points[i][1])
    area2 = (test_annotation['End_X'][i] - test_annotation['Start_X'][i]) * (test_annotation['End_Y'][i] - test_annotation['Start_Y'][i])
    X_diff = max(0, min(bounding_box_points[i][2], test_annotation['End_X'][i]) - max(bounding_box_points[i][0], test_annotation['Start_X'][i]))
    Y_diff = max(0, min(bounding_box_points[i][3], test_annotation['End_Y'][i]) - max(bounding_box_points[i][1], test_annotation['Start_Y'][i]))
    overlapp = X_diff * Y_diff
    total_overlapp += overlapp
    area = area + area1 + area2 - overlapp
print("Segmentation accuracy for test set:", total_overlapp/area)

Segmentation accuracy for test set: 0.763304833157623


In [25]:
print("Extracting test features")
test_set_images = []
test_set_images_ground = []
test_set_cat = []
test_set_features = []
test_set_features_ground = []
for i in range(len(test_images)):
    if(bounding_box_points[i][0] != 0 and bounding_box_points[i][1] != 0 and bounding_box_points[i][2] != 0 and bounding_box_points[i][3] != 0):
        test_set_images.append(test_images[i][bounding_box_points[i][1]:bounding_box_points[i][3], bounding_box_points[i][0]:bounding_box_points[i][2]])
    else:
        test_set_images.append(test_images[i])
    test_set_images_ground.append(test_images[i][test_annotation['Start_Y'][i]:test_annotation['End_Y'][i], test_annotation['Start_X'][i]:test_annotation['End_X'][i]])
    test_set_cat.append(test_category[i])
    test_set_features.append(feature_extraction(test_set_images[i]))
    test_set_features_ground.append(feature_extraction(test_set_images_ground[i]))
test_set_features = scaler.transform(test_set_features)
test_set_features_ground = scaler.transform(test_set_features_ground)

print("Predicting test set...")
predicted_test_classes = classifier.predict(test_set_features)
predicted_test_ground_classes = classifier.predict(test_set_features_ground)
# print(predicted_test_classes)
print("Calculating test set accuracy...")
test_accuracy = accuracy_score(test_set_cat, predicted_test_classes)
test_accuracy_ground = accuracy_score(test_set_cat, predicted_test_ground_classes)
print("Correct prediction - with proposed segmentation method: ", int(len(predicted_test_classes) * test_accuracy), "/", len(predicted_test_classes))
print("Accuracy score - with proposed segmentation method: ", test_accuracy)
print("Correct prediction - ground: ", int(len(predicted_test_ground_classes) * test_accuracy_ground), "/", len(predicted_test_classes))
print("Accuracy score - ground: ", test_accuracy_ground)

Extracting test features
Predicting test set...
Calculating test set accuracy...
Correct prediction - with proposed segmentation method:  1404 / 1994
Accuracy score - with proposed segmentation method:  0.7041123370110332
Correct prediction - ground:  1920 / 1994
Accuracy score - ground:  0.9628886659979939
